# Template Auditing Tool - TAT

The point of this project is to provide a tool for finding Unicode characters in templates. We can drop in a .csv and get a report of which templates have Unicode characters. This tool compares the length of a template message in characters vs. the length in bytes, as demonstrated in the code cell below. If character length and byte length differ, we know there are Unicode characters.

In [16]:
print(len('résumé')) # 6 characters
print(len('résumé'.encode('utf-8'))) # 8 bytes

6
8


2.0 templates are encoded as UTF-8.<br>
1.0 templates are ANSI.

Confirmed by opening the [.csv dumps](https://app.hubspot.com/contacts/21880972/record/0-5/1722173983/view/0?engagement=36665045903) in Notepad++, which displays the encoding on the far right of the bottom bar.

In [44]:
# We'll use pandas for data wrangling
import pandas as pd

# Replace the .csv below as needed
# Make sure to specify the correct encoding
# UTF-8 for 2.0 Templates
# ANSI for 1.0 Templates

# templates = pd.read_csv("../T2C1.0_TemplateMessagesReport_07.03.csv", encoding='ANSI')
templates = pd.read_csv("../2_0Templates.csv", encoding='UTF-8')

templates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5231 entries, 0 to 5230
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   AccountId    5231 non-null   object
 1   AccountName  5231 non-null   object
 2   TemplateId   5231 non-null   object
 3   Name         5231 non-null   object
 4   OrgUnitName  5231 non-null   object
 5   Description  597 non-null    object
 6   Message      5231 non-null   object
dtypes: object(7)
memory usage: 286.2+ KB


In [45]:
# View the first three rows
# to understand columns and their data
templates.head(3)

,AccountId,AccountName,TemplateId,Name,OrgUnitName,Description,Message
0,a26a48e7-6d51-412f-b36e-d4dd4a0e33c2,"CPCWA Co., LLLP dba Power Finance Texas",1,Customer Portal Registration,Account Group,NaN,Power Finance Tx: Follow this link to finalize...
1,a26a48e7-6d51-412f-b36e-d4dd4a0e33c2,"CPCWA Co., LLLP dba Power Finance Texas",4,Doc Signature Request,Account Group,NaN,Power Finance Tx: Follow this link to setup a ...
2,a26a48e7-6d51-412f-b36e-d4dd4a0e33c2,"CPCWA Co., LLLP dba Power Finance Texas",5,Thanks for Pmt 1,Account Group,NaN,Power Finance Tx: Thanks for your pmt of {Amou...


In [46]:
# Check the output above, but as of this writing,
# 2.0 template messages are in the "Message" column
# 1.0 templates messages are in the "TextMessage" column
if 'Message' in templates.columns:
    # messages = templates["Message"] # 2.0 Templates
    templates["Msg_Characters"] = templates.Message.str.len()
    templates["Msg_Bytes"] = templates["Message"].apply(lambda x: len(x.encode('utf-8')))
else:
    # messages = templates.iloc[:, 6]
    # messages = templates["TextMessage"]

    # For consistency, put message body in Message column
    templates["Message"] = templates["TextMessage"]

    templates["Msg_Characters"] = templates.TextMessage.str.len()
    templates["Msg_Bytes"] = templates["TextMessage"].apply(lambda x: len(x.encode('utf-8')))

templates.head(3)

,AccountId,AccountName,TemplateId,Name,OrgUnitName,Description,Message,Msg_Characters,Msg_Bytes
0,a26a48e7-6d51-412f-b36e-d4dd4a0e33c2,"CPCWA Co., LLLP dba Power Finance Texas",1,Customer Portal Registration,Account Group,NaN,Power Finance Tx: Follow this link to finalize...,114,114
1,a26a48e7-6d51-412f-b36e-d4dd4a0e33c2,"CPCWA Co., LLLP dba Power Finance Texas",4,Doc Signature Request,Account Group,NaN,Power Finance Tx: Follow this link to setup a ...,119,119
2,a26a48e7-6d51-412f-b36e-d4dd4a0e33c2,"CPCWA Co., LLLP dba Power Finance Texas",5,Thanks for Pmt 1,Account Group,NaN,Power Finance Tx: Thanks for your pmt of {Amou...,92,92


In [47]:
unicode_detected = templates.query('Msg_Characters != Msg_Bytes')

unicode_detected

,AccountId,AccountName,TemplateId,Name,OrgUnitName,Description,Message,Msg_Characters,Msg_Bytes
41,8cb25aae-c2dc-49af-aef1-62fa8687a0cf,RSI Enterprises Inc.,36,2-Way Incorrect #,RSI Enterprises Group,2-Way,Please reply STOP to this message to prevent f...,93,95
981,da75a7ee-3a66-4854-8284-102f97381fc2,Newport Acceptance Corp,14,PTP 7 Day ENG,Account Group,NaN,Newport Acceptance: We’d like to discuss your...,157,159
982,da75a7ee-3a66-4854-8284-102f97381fc2,Newport Acceptance Corp,16,Avoid Late Fee ENG,Account Group,NaN,Newport Acceptance: We’d like to discuss your...,157,159
983,da75a7ee-3a66-4854-8284-102f97381fc2,Newport Acceptance Corp,18,30DQ ENG,Account Group,NaN,Newport Acceptance: We’d like to discuss your...,157,159
984,da75a7ee-3a66-4854-8284-102f97381fc2,Newport Acceptance Corp,20,60DQ ENG,Account Group,NaN,Newport Acceptance: We’d like to discuss your...,157,159
...,...,...,...,...,...,...,...,...,...
5138,31826c31-c1a0-4d3e-8872-016bb1131ed4,"Credits, Inc.",2,No Contact 1,"Credits, Inc Group",NaN,"Credits, Inc. - Debt Collector: Account (Acct#...",179,183
5139,31826c31-c1a0-4d3e-8872-016bb1131ed4,"Credits, Inc.",3,No Contact 2,"Credits, Inc Group",NaN,"Credits, Inc. - Debt Collector is here to help...",187,191
5145,31826c31-c1a0-4d3e-8872-016bb1131ed4,"Credits, Inc.",9,Past Due: 1 Day,"Credits, Inc Group",NaN,"Credits, Inc. - Debt Collector: Don't forget a...",255,263
5146,31826c31-c1a0-4d3e-8872-016bb1131ed4,"Credits, Inc.",10,Past Due: 5 Day,"Credits, Inc Group",NaN,"Credits, Inc. - Debt Collector: We understand ...",245,253


In [89]:
# GSM 7 Char Set
# These count as one character
gsm_char_set = {'@', 'Δ', ' ', '0', '¡', 'P', '¿', 'p', '£', '_', '!', '1', 'A', 'Q', 'a', 'q', '$', 'Φ', '"', '2', 'B', 'R', 'b', 'r', '¥', 'Γ', '#', '3', 'C', 'S', 'c', 's', 'è', 'Λ', '¤', '4', 'D', 'T', 'd', 't', 'é', 'Ω', '%', '5', 'E', 'U', 'e', 'u', 'ù', 'Π', '&', '6', 'F', 'V', 'f', 'v', 'ì', 'Ψ', '‘', '7', 'G', 'W', 'g', 'w', 'ò', 'Σ', '(', '8', 'H', 'X', 'h', 'x', 'Ç', 'Θ', ')', '9', 'I', 'Y', 'i', 'y', 'LF', 'Ξ', '*', ':', 'J', 'Z', 'j', 'z', 'Ø', 'ESC', '+', ';', 'K', 'Ä', 'k', 'ä', 'ø', 'Æ', ',', '<', 'L', 'Ö', 'l', 'ö', 'CR', 'æ', '-', '=', 'M', 'Ñ', 'm', 'ñ', 'Å', 'ß', '.', '>', 'N', 'Ü', 'n', 'ü', 'å', 'É', '/', '?', 'O', '§', 'o', 'à'}

# GSM 7 Extended Set
# these require escape (\) and count as two characters
gsm_extended_set = {'^', '€', '|', '{', '}', '[', ']', '~'}

# gsm_set = gsm_char_set.union(gsm_extended_set)

# gsm_set_str = "@ £ $ ¥ è é ù ì ò Ç  Ø ø  Å å Δ _ Φ Γ Λ Ω Π Ψ Σ Θ Ξ   ^ { } \ [ ~ ] | € Æ æ ß É  ! \" # ¤ % & ' ( ) * + , - . / 0 1 2 3 4 5 6 7 8 9 : ; < = > ? ¡ A B C D E F G H I J K L M N O P Q R S T U V W X Y Z Ä Ö Ñ Ü § ¿ a b c d e f g h i j k l m n o p q r s t u v w x y z ä ö ñ ü à"
# gsm_list = gsm_set_str.split(' ')
# # print(gsm_list)
# gsm_set = set(gsm_list).union({" "})
gsm_set = {'t', '%', 'W', 'L', 'Q', '¥', ')', 'N', 'ß', 'o', '&', 'c', '4', 'I', 'é', 'Φ', 'n', 'G', 'Π', '€', 'X', '>', 'f', 'à', 's', 'm', 'l', '<', 'd', 'O', 'p', '#', '~', 'ñ', 'Δ', 'M', 'x', '*', '$', 'Ü', '9', 'P', 'Ø', 'Σ', 'Ξ', 'E', 'i', 'Ψ', '1', ',', 'ì', '(', 'Θ', 'Λ', 'F', '_', '@', '/', 'h', 'V', 'ä', '+', ']', 'z', 'a', 'D', 'T', '"', 'e', 'É', ':', '!', 'k', 'w', 'ù', 'Ä', 'g', '£', '0', '6', '§', 'j', 'Y', 'H', ' ', '|', '¤', '-', 'Æ', 'J', 'y', '{', '?', '^', '2', 'S', '¿', ';', '=', 'ö', 'K', 'b', '5', '.', 'å', 'ü', '7', 'æ', 'U', 'Å', '\\', 'Γ', 'ò', 'Ω', '¡', 'u', 'ø', "'", '}', 'B', 'r', 'Z', 'è', '3', 'Ö', 'A', '8', 'Ç', 'v', 'q', '[', 'Ñ', 'R', 'C'}

print(gsm_set)
print(type(gsm_set))

def findNonGsm(str):
    # Convert String to Set
    str_set = set(str)

    diff = str_set.difference(gsm_set)

    if len(diff) == 0:
        return "NONE"
    else:
        return diff

# print(findNonGsm('Please reply STOP to this message to prevent future texts. We apologize for the wrong number.'))

{'M', 'e', '¡', 'a', 'D', 'R', 'J', 'W', ':', '?', 'U', '"', 'à', '9', 'x', 'ò', "'", '~', 'Θ', 'B', 'Σ', 'b', '\\', '[', 'Å', '#', 'X', 'ü', 'ì', 'u', 'ß', 'h', 'l', '4', 'O', 'n', 'G', 'T', '&', 'L', 'S', 'A', '*', 'è', '=', 'z', '7', 'F', '|', 'ø', 'i', '5', 'w', '¤', 'É', '{', 'K', 'Π', 'd', 'Ü', 'Ñ', 'p', 'Δ', 'Λ', 'Ω', '%', 'Z', '2', 'H', 'o', '^', '£', '+', 'N', '$', 'g', 'V', 'ö', '8', 'v', 'j', 'm', 'I', '>', '(', ')', 'Ø', ' ', '-', 'k', '¥', 'C', 't', '1', '¿', 'y', '}', 'Φ', 'ä', 'Ö', 'ñ', '@', '0', 'ù', '6', ';', 'Ä', '§', 'Q', 'Y', '!', '3', ']', 'P', ',', 'r', 'f', 'E', '€', 'c', 'Æ', '_', 'Γ', '.', 's', 'q', 'æ', 'å', '/', 'Ψ', '<', 'Ç', 'Ξ', 'é'}
<class 'set'>


In [90]:
templates["NonGSM_Chars"] = templates["Message"].apply(lambda x: findNonGsm(x))

templates

,AccountId,AccountName,TemplateId,Name,OrgUnitName,Description,Message,Msg_Characters,Msg_Bytes,NonGSM_Chars
0,a26a48e7-6d51-412f-b36e-d4dd4a0e33c2,"CPCWA Co., LLLP dba Power Finance Texas",1,Customer Portal Registration,Account Group,NaN,Power Finance Tx: Follow this link to finalize...,114,114,NONE
1,a26a48e7-6d51-412f-b36e-d4dd4a0e33c2,"CPCWA Co., LLLP dba Power Finance Texas",4,Doc Signature Request,Account Group,NaN,Power Finance Tx: Follow this link to setup a ...,119,119,NONE
2,a26a48e7-6d51-412f-b36e-d4dd4a0e33c2,"CPCWA Co., LLLP dba Power Finance Texas",5,Thanks for Pmt 1,Account Group,NaN,Power Finance Tx: Thanks for your pmt of {Amou...,92,92,NONE
3,a26a48e7-6d51-412f-b36e-d4dd4a0e33c2,"CPCWA Co., LLLP dba Power Finance Texas",6,Thanks for Pmt 2,Account Group,NaN,Power Finance Tx: Thanks for your pmt of {Amou...,137,137,NONE
4,a26a48e7-6d51-412f-b36e-d4dd4a0e33c2,"CPCWA Co., LLLP dba Power Finance Texas",7,Re-Open Account,Account Group,NaN,Power Finance Tx: Thanks for your pmt of {Amou...,141,141,NONE
...,...,...,...,...,...,...,...,...,...,...
5226,3ecd0e57-bf78-459b-9ae0-af45d8380efc,Quick Collect Inc,5,No Contact 3,Quick Collect Group,NaN,"Quick Collect, Inc.-Debt Collector: Please cal...",138,138,NONE
5227,3ecd0e57-bf78-459b-9ae0-af45d8380efc,Quick Collect Inc,6,Offer 1,Quick Collect Group,NaN,"Quick Collect, Inc.-Debt Collector: We are try...",150,150,NONE
5228,3ecd0e57-bf78-459b-9ae0-af45d8380efc,Quick Collect Inc,7,Offer 2,Quick Collect Group,NaN,"Quick Collect, Inc.-Debt Collector has an oppo...",153,153,NONE
5229,3ecd0e57-bf78-459b-9ae0-af45d8380efc,Quick Collect Inc,8,Offer 3,Quick Collect Group,NaN,We can REMOVE up to {Perc}% off your balance w...,158,158,NONE


In [91]:
nonGSM_detected = templates[templates["NonGSM_Chars"] != "NONE"]

nonGSM_detected

,AccountId,AccountName,TemplateId,Name,OrgUnitName,Description,Message,Msg_Characters,Msg_Bytes,NonGSM_Chars
41,8cb25aae-c2dc-49af-aef1-62fa8687a0cf,RSI Enterprises Inc.,36,2-Way Incorrect #,RSI Enterprises Group,2-Way,Please reply STOP to this message to prevent f...,93,95,{ }
981,da75a7ee-3a66-4854-8284-102f97381fc2,Newport Acceptance Corp,14,PTP 7 Day ENG,Account Group,NaN,Newport Acceptance: We’d like to discuss your...,157,159,{’}
982,da75a7ee-3a66-4854-8284-102f97381fc2,Newport Acceptance Corp,16,Avoid Late Fee ENG,Account Group,NaN,Newport Acceptance: We’d like to discuss your...,157,159,{’}
983,da75a7ee-3a66-4854-8284-102f97381fc2,Newport Acceptance Corp,18,30DQ ENG,Account Group,NaN,Newport Acceptance: We’d like to discuss your...,157,159,{’}
984,da75a7ee-3a66-4854-8284-102f97381fc2,Newport Acceptance Corp,20,60DQ ENG,Account Group,NaN,Newport Acceptance: We’d like to discuss your...,157,159,{’}
...,...,...,...,...,...,...,...,...,...,...
5138,31826c31-c1a0-4d3e-8872-016bb1131ed4,"Credits, Inc.",2,No Contact 1,"Credits, Inc Group",NaN,"Credits, Inc. - Debt Collector: Account (Acct#...",179,183,"{”, “}"
5139,31826c31-c1a0-4d3e-8872-016bb1131ed4,"Credits, Inc.",3,No Contact 2,"Credits, Inc Group",NaN,"Credits, Inc. - Debt Collector is here to help...",187,191,"{”, “}"
5145,31826c31-c1a0-4d3e-8872-016bb1131ed4,"Credits, Inc.",9,Past Due: 1 Day,"Credits, Inc Group",NaN,"Credits, Inc. - Debt Collector: Don't forget a...",255,263,"{”, “}"
5146,31826c31-c1a0-4d3e-8872-016bb1131ed4,"Credits, Inc.",10,Past Due: 5 Day,"Credits, Inc Group",NaN,"Credits, Inc. - Debt Collector: We understand ...",245,253,"{”, “}"


In [93]:
# Convert NonGSM_Chars from set to string using repr
nonGSM_detected['repr'] = nonGSM_detected['NonGSM_Chars'].apply(lambda x: repr(x))

nonGSM_detected.info()
nonGSM_detected.head()


<class 'pandas.core.frame.DataFrame'>
Index: 638 entries, 41 to 5183
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   AccountId       638 non-null    object
 1   AccountName     638 non-null    object
 2   TemplateId      638 non-null    object
 3   Name            638 non-null    object
 4   OrgUnitName     638 non-null    object
 5   Description     13 non-null     object
 6   Message         638 non-null    object
 7   Msg_Characters  638 non-null    int64 
 8   Msg_Bytes       638 non-null    int64 
 9   NonGSM_Chars    638 non-null    object
 10  repr            638 non-null    object
dtypes: int64(2), object(9)
memory usage: 59.8+ KB


C:\Users\JarvisMartin\AppData\Local\Temp\ipykernel_6148\1453203699.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonGSM_detected['repr'] = nonGSM_detected['NonGSM_Chars'].apply(lambda x: repr(x))


,AccountId,AccountName,TemplateId,Name,OrgUnitName,Description,Message,Msg_Characters,Msg_Bytes,NonGSM_Chars,repr
41,8cb25aae-c2dc-49af-aef1-62fa8687a0cf,RSI Enterprises Inc.,36,2-Way Incorrect #,RSI Enterprises Group,2-Way,Please reply STOP to this message to prevent f...,93,95,{ },{'\xa0'}
981,da75a7ee-3a66-4854-8284-102f97381fc2,Newport Acceptance Corp,14,PTP 7 Day ENG,Account Group,NaN,Newport Acceptance: We’d like to discuss your...,157,159,{’},{'’'}
982,da75a7ee-3a66-4854-8284-102f97381fc2,Newport Acceptance Corp,16,Avoid Late Fee ENG,Account Group,NaN,Newport Acceptance: We’d like to discuss your...,157,159,{’},{'’'}
983,da75a7ee-3a66-4854-8284-102f97381fc2,Newport Acceptance Corp,18,30DQ ENG,Account Group,NaN,Newport Acceptance: We’d like to discuss your...,157,159,{’},{'’'}
984,da75a7ee-3a66-4854-8284-102f97381fc2,Newport Acceptance Corp,20,60DQ ENG,Account Group,NaN,Newport Acceptance: We’d like to discuss your...,157,159,{’},{'’'}


In [112]:
# Check for NBSP
char = 'xa0'
nonGSM_detected['search'] = nonGSM_detected['repr'].str.contains(char)

search_results = nonGSM_detected[nonGSM_detected['search'] == True]
search_results.info()
search_results.head()

<class 'pandas.core.frame.DataFrame'>
Index: 23 entries, 41 to 5183
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   AccountId       23 non-null     object
 1   AccountName     23 non-null     object
 2   TemplateId      23 non-null     object
 3   Name            23 non-null     object
 4   OrgUnitName     23 non-null     object
 5   Description     13 non-null     object
 6   Message         23 non-null     object
 7   Msg_Characters  23 non-null     int64 
 8   Msg_Bytes       23 non-null     int64 
 9   NonGSM_Chars    23 non-null     object
 10  repr            23 non-null     object
 11  search          23 non-null     bool  
dtypes: bool(1), int64(2), object(9)
memory usage: 2.2+ KB


C:\Users\JarvisMartin\AppData\Local\Temp\ipykernel_6148\3285671049.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonGSM_detected['search'] = nonGSM_detected['repr'].str.contains(char)


,AccountId,AccountName,TemplateId,Name,OrgUnitName,Description,Message,Msg_Characters,Msg_Bytes,NonGSM_Chars,repr,search
41,8cb25aae-c2dc-49af-aef1-62fa8687a0cf,RSI Enterprises Inc.,36,2-Way Incorrect #,RSI Enterprises Group,2-Way,Please reply STOP to this message to prevent f...,93,95,{ },{'\xa0'},True
1196,9083fcb0-d50e-4b6f-a928-ed940a4030e2,Holloway Credit Solutions,39,2-Way Incorrect #,Holloway Account Group,2-Way,Please reply STOP to this message to prevent f...,93,95,{ },{'\xa0'},True
1235,9083fcb0-d50e-4b6f-a928-ed940a4030e2,Holloway Credit Solutions,21,Offer 4,Holloway Account Group,NaN,"HCS, a Debt Collector Special Offer off your {...",151,152,{ },{'\xa0'},True
1257,9083fcb0-d50e-4b6f-a928-ed940a4030e2,Holloway Credit Solutions,214,2-Way Incorrect #,Atlanticare Group,2-Way,Please reply STOP to this message to prevent f...,93,95,{ },{'\xa0'},True
1317,9083fcb0-d50e-4b6f-a928-ed940a4030e2,Holloway Credit Solutions,155,2-Way Incorrect #,Atlanticare SP Group,2-Way,Responda PARE a este mensaje para evitar futur...,111,113,{ },{'\xa0'},True


In [ ]:
# create csv
search_results.to_excel('search_results.xlsx')

In [52]:
# Read in Msg Volume by Template ID CSV as dataframe
# msg_vol = pd.read_csv("../../Attachments/T2C1.0_MsgsVolume_ByTemplateID_June2023.csv", encoding='ANSI')
msg_vol = pd.read_csv("../CommonData.TemplateMessageCount.csv", encoding='ANSI')

msg_vol.info()

# View the first 3
msg_vol.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   AccountName  265 non-null    object
 1   AccountId    265 non-null    object
 2   TemplateId   265 non-null    int64 
 3   count        265 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 8.4+ KB


,AccountName,AccountId,TemplateId,count
0,"CPCWA Co., LLLP dba Power Finance Texas",a26a48e7-6d51-412f-b36e-d4dd4a0e33c2,3,12205
1,"CPCWA Co., LLLP dba Power Finance Texas",a26a48e7-6d51-412f-b36e-d4dd4a0e33c2,4,4641
2,"CPCWA Co., LLLP dba Power Finance Texas",a26a48e7-6d51-412f-b36e-d4dd4a0e33c2,2,3140


In [59]:
# For 2.0, convert TemplateId to string for merge
msg_vol['TemplateId'] = msg_vol['TemplateId'].astype('str')

msg_vol.dtypes

AccountName    object
AccountId      object
TemplateId     object
count           int64
dtype: object

In [60]:
# Use CompanyName and TemplateID to cross-reference (1.0 Only?)
# msg_vol_and_nonGSM_detected = pd.merge(
#     msg_vol, nonGSM_detected, on=["CompanyName", "TemplateID"]
# )

# For 2.0
msg_vol_and_nonGSM_detected = pd.merge(
    msg_vol, nonGSM_detected, on=["AccountId", "TemplateId"]
)

msg_vol_and_nonGSM_detected.info()
msg_vol_and_nonGSM_detected.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   AccountName_x   2 non-null      object
 1   AccountId       2 non-null      object
 2   TemplateId      2 non-null      object
 3   count           2 non-null      int64 
 4   AccountName_y   2 non-null      object
 5   Name            2 non-null      object
 6   OrgUnitName     2 non-null      object
 7   Description     1 non-null      object
 8   Message         2 non-null      object
 9   Msg_Characters  2 non-null      int64 
 10  Msg_Bytes       2 non-null      int64 
 11  NonGSM_Chars    2 non-null      object
dtypes: int64(3), object(9)
memory usage: 324.0+ bytes


,AccountName_x,AccountId,TemplateId,count,AccountName_y,Name,OrgUnitName,Description,Message,Msg_Characters,Msg_Bytes,NonGSM_Chars
0,Gateway Financial,98cc075a-9205-4033-9220-c5bd80001682,708,1,Gateway Financial,Qualified Deferment,Gateway Collection Group,Qualified Deferment,GFS: YOU MAY QUALIFY FOR A PAYMENT DEFERMENT! ...,166,167,{ }
1,"Integrated Delivery Solutions, LLC",15ed25b8-4a60-407e-b409-9b0d9e7e9152,6,5426,"Integrated Delivery Solutions, LLC",Loan Request,Marketing Group,NaN,"{first_name}, thank you for your request for a...",202,205,{ }


In [43]:
# create csv
# msg_vol_and_nonGSM_detected.to_csv('template_audit.csv', index=False, header=True)
msg_vol_and_nonGSM_detected.to_excel('template_audit.xlsx')

In [ ]:
# Unnecessary
# Experiment

from pandas import merge_ordered

# Use CompanyName and TemplateID to cross-reference (1.0 Only?)
ordered_merge = merge_ordered(msg_vol, nonGSM_detected, fill_method="ffill", left_by="June'2023_MsgVolume")

ordered_merge.info()
ordered_merge.head()

False Positive
RSI Enterprises Group, Template #36

non-GSM set shows {}, but is fine in calculator.